In [57]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from dateutil.parser import parse

In [3]:
entso_e_token = 'd9281f86-65db-4158-b487-4eeb317c4f2e'
entso_e_url = 'https://transparency.entsoe.eu/api'

In [156]:
grid = {
    'France, RTE BZ / CA / MBA': '10YFR-RTE------C',
    'Denmark': '10Y1001A1001A65H',
    'Czech Republic, CEPS BZ / CA/ MBA': '10YCZ-CEPS-----N',
    'Poland, PSE SA BZ / BZA / CA / MBA': '10YPL-AREA-----S',
    # 'Norway, Norway MBA, Stattnet CA': '10YNO-0--------C',
    'Austria, APG CA / MBA': '10YAT-APG------L',
    'Luxembourg, CREOS CA': '10YLU-CEGEDEL-NQ',
    # 'Belgium, Elia BZ / CA / MBA': '10YBE----------2'
    'Netherlands, TenneT NL BZ / CA/ MBA': '10YNL----------L'
}

In [157]:
def get_germany_inflow(start_year, end_year):

    in_data = pd.DataFrame()
    for grid_elem in grid.keys():
        grid_elem_data = pd.DataFrame()
        for year in list(range(start_year, end_year+1)):
            params = {
                'securityToken': entso_e_token,
                'documentType': 'A11',
                'in_Domain': '10Y1001A1001A83F',
                'out_Domain': grid[grid_elem],
                'periodStart': '{}01010000'.format(year),
                'periodEnd': '{}01010000'.format(year+1)
            }

            r = requests.get(entso_e_url, params)
            root = ET.fromstring(r.content)

            code = '{urn:iec62325.351:tc57wg16:451-3:publicationdocument:7:0}'
            for ts in root.findall(code+'TimeSeries'):
                start = ts.find(code+'Period').find(code+'timeInterval').find(code+'start').text
                end = ts.find(code+'Period').find(code+'timeInterval').find(code+'end').text
                points = ts.find(code+'Period').findall(code+'Point')
                df = pd.DataFrame(
                    [int(pt.find(code+'quantity').text) for pt in points],
                    index=pd.date_range(start=parse(start), end=parse(end), freq='1H', closed='left'),
                    columns=[grid_elem]
                )
                grid_elem_data = pd.concat([grid_elem_data, df], axis=0, verify_integrity=True)

        in_data = pd.concat([in_data, grid_elem_data], axis=1, verify_integrity=True)
    
    return in_data

In [160]:
get_germany_inflow(2021, 2021)

,"France, RTE BZ / CA / MBA",Denmark,"Czech Republic, CEPS BZ / CA/ MBA","Poland, PSE SA BZ / BZA / CA / MBA","Austria, APG CA / MBA","Luxembourg, CREOS CA","Netherlands, TenneT NL BZ / CA/ MBA"
2021-01-01 00:00:00+00:00,0,2701,2165,0,846,0,0
2021-01-01 01:00:00+00:00,0,2473,2070,0,948,0,0
2021-01-01 02:00:00+00:00,40,2406,2305,0,951,0,0
2021-01-01 03:00:00+00:00,437,2328,2362,0,910,0,0
2021-01-01 04:00:00+00:00,158,2306,2317,0,877,0,0
...,...,...,...,...,...,...,...
2021-03-20 12:00:00+00:00,225,1784,0,0,8,0,2434
2021-03-20 13:00:00+00:00,569,1725,0,0,24,0,2131
2021-03-20 14:00:00+00:00,1217,1628,2,0,44,0,2036
2021-03-20 15:00:00+00:00,1746,1825,146,0,20,0,1799
